Importando as bibliotecas

In [ ]:
import pandas as pd
import requests
import json

Obtemos a tabela da Wikipedia diretamente em um DataFrame

In [ ]:
data = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]


Separamos os códigos dos endereços e eliminamos aqueles que não são atribuídos

In [ ]:
postal_code = []
status = []
for row in data.iterrows():
    for i in row[1]:
        postal_code.append(i[:3])
        status.append(i[3:])

In [ ]:
inte = pd.DataFrame({"Postal Code":postal_code,"Status":status})
inte = inte[inte.Status != "Not assigned"]

Separamos o Bairro dos endereços. Em seguida, limpamos o DataFrame e corrigimos alguns erros de digitação

In [ ]:
inte["Borough"] = [i[0] for i in inte.Status.str.split("(")]
inte["Neighborhood"] = [",".join(i[1][:-1].split("/")) for i in inte.Status.str.split("(")]
inte.reset_index(inplace = True)
inte.drop(["index","Status"],axis = 1, inplace = True)

In [ ]:
inte["Neighborhood"].replace(["Don Mills)Nort","Don Mills)Sout","Downsview)East","Downsview)Wes","Downsview)Centra","Willowdale)Sout","Downsview)Northwes","Willowdale)Wes"],
                             ["Don Mills North","Don Mills South","Downsview East","Downsview West","Downsview Central","Willowdale South","Downsview Northwest","Willowdale West"],
                             inplace = True)

In [ ]:
inte["Borough"].replace(["East YorkEast Toronto","Mississauga, Canada Post Gateway Processing Centre","Downtown TorontoStn A PO Boxes25 The Esplanade","EtobicokeNorthwest","East TorontoBusiness reply mail Processing Centre969 Eastern"],
                        ["East York, East Toronto","Mississauga, Canada Post Gateway Processing Centre","Downtown Toronto, Stn A PO Boxes, 25 The Esplanade","Etobicoke Northwest","East Toronto, Business reply mail Processing Centre, 969 Eastern"],
                        inplace = True)

Resultado

In [ ]:
inte.head()

Postal Code           Borough                       Neighborhood
0         M3A        North York                          Parkwoods
1         M4A        North York                   Victoria Village
2         M5A  Downtown Toronto         Regent Park , Harbourfront
3         M6A        North York  Lawrence Manor , Lawrence Heights
4         M7A      Queen's Park      Ontario Provincial Government

In [ ]:
inte.shape

(103, 3)

Carregamos o ficheiro .CSV

In [ ]:
geo_data = pd.read_csv("Geospatial_Coordinates.csv",header = 0)
geo_data.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

Juntamos os dois DataFrames usando o código postal como chave

In [ ]:
df = inte.merge(geo_data, on = "Postal Code")
df.head()

Postal Code           Borough                       Neighborhood   Latitude  \
0         M3A        North York                          Parkwoods  43.753259   
1         M4A        North York                   Victoria Village  43.725882   
2         M5A  Downtown Toronto         Regent Park , Harbourfront  43.654260   
3         M6A        North York  Lawrence Manor , Lawrence Heights  43.718518   
4         M7A      Queen's Park      Ontario Provincial Government  43.662301   

   Longitude  
0 -79.329656  
1 -79.315572  
2 -79.360636  
3 -79.464763  
4 -79.389494

Resultado

In [ ]:
df.shape

(103, 5)